In [ ]:
import ipywidgets as widgets
from IPython.display import display, Video, HTML, clear_output
from ipywidgets import Button, HBox, Layout
import asyncio

from collections.abc import Collection

print(widgets.__version__)

# print([method_name for method_name in dir(object)
#                   if callable(getattr(object, method_name))])

speed = widgets.FloatSlider(
    min=0.1,
    max=2,
    step=0.1,
    description='Playbackspeed:',
    value=1
)

display(speed)

In [ ]:
timestamp = 2.0
window_size = 2


def display_html(timestamp, window_size):
    print(timestamp, window_size)
    display(HTML(f'''
        <body>
            <video id="window" width="500px"; class="center" src="Timer2.mp4"></video><br>
            <button onclick="play()">Play</button>
            <button onclick="pause()">Pause</button>
            <script type="text/javascript">
                var timestamp = {timestamp}
                var window_size = {window_size}

                function play(){{
                    var video= document.getElementById("window");
                    if (video.currentTime < timestamp || video.currentTime >= timestamp + window_size){{
                        video.currentTime = timestamp;
                    }}
                    video.play()
                    setInterval(function(){{
                        if(video.currentTime >= timestamp + window_size){{
                            video.currentTime = timestamp;
                        }}
                    }},1);
                }}
                
                function pause(){{
                    var video= document.getElementById("window");
                    video.pause();
                }}
            </script>
        </body>
    '''))

# Here you can enter your activities
activities = ["running", "walking", "biking"]

switch = [Button(description=(name +' ' + str(i))) for i, name in enumerate(activities)]

combined = HBox([items for items in switch], layout = Layout(flex_flow='row wrap'))

def upon_clicked(btn):
    print(f'Classified as {btn.description}.')
    for n in range(len(activities)):
        switch[n].style.button_color = 'white'
    btn.style.button_color = 'gray'
    
# while True:
#     if keyboard.is_pressed("a"):
#         print("You pressed 'a'.")
#         break

for n in range(len(activities)):
    switch[n].on_click(upon_clicked)

display_html(timestamp, window_size)
display(combined)

In [ ]:
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

from ipywidgets import IntSlider, Output
slider = IntSlider()
out = Output()

async def f():
    for i in range(10):
        out.append_stdout('did work ' + str(i) + '\n')
        x = await wait_for_change(slider, 'value')
        print(x)
        out.append_stdout('async function continued with value ' + str(x) + '\n')
asyncio.ensure_future(f())

slider



In [ ]:
class VideoLabeler():
    def __init__(self, labels: Collection[str]) -> None:
        self.labels = labels
        self.new_label = ''
        self.switch = [Button(description=name) for name in self.labels]

    def labeling(self, video_file, timestamp, window_size) -> str:
        # print(f'Current timestamp: {timestamp}')
        self.display_html(video_file, timestamp, window_size)
        print(f'Enter the index or the name of one of the following labels. Enter x to add a new label:')
        for i, label in enumerate(self.labels):
            print(f'{i}. {label}')
        new_label = input()
        return new_label

    def display_html(self, video_file, timestamp, window_size) -> None:
        display(HTML(f'''
            <body>
                <video id="window" width="500px" src="{video_file}"></video><br>
                <button onclick="play()">Play</button>
                <button onclick="pause()">Pause</button>
                <script type="text/javascript">
                    var timestamp = {timestamp}
                    var window_size = {window_size}

                    function play(){{
                        var video= document.getElementById("window");
                        if (video.currentTime < timestamp || video.currentTime >= timestamp + window_size){{
                            video.currentTime = timestamp;
                        }}
                        video.play()
                        setInterval(function(){{
                            if(video.currentTime >= timestamp + window_size){{
                                video.currentTime = timestamp;
                            }}
                        }},1);
                        <!--video.playbackRate = 2-->
                    }}
                
                    function pause(){{
                        var video= document.getElementById("window");
                        video.pause();
                    }}
                </script>
            </body>
        '''))

    def display_buttons(self):
        combined = HBox([items for items in self.switch], layout = Layout(flex_flow='row wrap'))

        for n in range(len(self.labels)):
            self.switch[n].on_click(self.upon_clicked)
        
        # asyncio.ensure_future(self.f2(switch))

        display(combined)

    def upon_clicked(self, btn: Button) -> None:
        # clear_output(wait=False)
        # print(f'Current timestamp: {timestamp}')
        # self.display_html(timestamp)
        # self.display_buttons()
        # print(f"Previous classified as '{btn.description}'.")        

        for n in range(len(self.labels)):
            self.switch[n].style.button_color = 'white'
        btn.style.button_color = 'gray'
        
        self.new_label = btn.description

    @staticmethod
    async def f2(button):
        for i in range(10):
            out.append_stdout('did work ' + str(i) + '\n')
            x = await wait_for_change(button, 'value')
            print(x)
            out.append_stdout('async function continued with value ' + str(x) + '\n')


vid_lab = VideoLabeler(["Walking", "Running", "Stairs"])
print(vid_lab.labeling('Timer2.mp4', 4, 2))

In [ ]:
for btn in vid_lab.switch:
    btn.

In [ ]:
import asyncio
import ipython_blocking

async def wait_click(button, sleep=0.05):
    clicked = False
    def on_click(button):
        nonlocal clicked
        clicked = True
    button.on_click(on_click)
    async def runner():
        ctx = ipython_blocking.CaptureExecution(replay=True)
        with ctx:
            while True:
                await asyncio.sleep(sleep)
                if clicked:
                    break
                ctx.step() # handles all other messages that aren't 'execute_request' including widget value changes
    return await asyncio.create_task(runner())

button = Button(description='Click me')
display(button)

print('before await')
await wait_click(button=button)
print('after await')

In [ ]:
from functools import wraps
from ipywidgets import IntSlider, VBox, HTML

def yield_for_change(widget, attribute):
    """Pause a generator to wait for a widget change event.

    This is a decorator for a generator function which pauses the generator on yield
    until the given widget attribute changes. The new value of the attribute is
    sent to the generator and is the value of the yield.
    """
    def f(iterator):
        @wraps(iterator)
        def inner():
            i = iterator()
            def next_i(change):
                try:
                    i.send(change.new)
                except StopIteration as e:
                    widget.unobserve(next_i, attribute)
            widget.observe(next_i, attribute)
            # start the generator
            next(i)
        return inner
    return f

slider2=IntSlider()
# buttons=widgets.ToggleButtons(
#     options=['walking', 'running', 'stairs'],
#     description = 'Labels:',
#     disabled=False
# )
labels = ['Walking', 'Running', 'Stairs']
buttons = [widgets.ToggleButton(value=False, description=name) for name in labels]
combined = HBox([button for button in buttons], layout = Layout(flex_flow='row wrap'))

y = False

@yield_for_change(buttons[0], 'value')
def f1():
    global y
    # print('did work %s'%i)
    x = yield
    if not y:
        print('generator function continued with value %s'%x)
        # print(buttons.value)
        y = True
    return 1
@yield_for_change(buttons[1], 'value')
def f2():
    global y
    # print('did work %s'%i)
    x = yield
    if not y:
        print('generator function continued with value %s'%x)
        # print(buttons.value)
        y = True
    return 2
@yield_for_change(buttons[2], 'value')
def f3():
    global y
    # print('did work %s'%i)
    x = yield
    if not y:
        print('generator function continued with value %s'%x)
        # print(buttons.value)
        y = True
    return 3
f1()
f2()
f3()

display(combined)
# display(slider2)
# print(buttons.value)

In [15]:
def display_html(video_file, timestamp, window_size) -> None:
        display(HTML(f'''
            <body>
                <video id="window" width="500px" src="{video_file}" muted></video><br>
                <script>
                    var video= document.getElementById("window");
                    video.currentTime = 2;
                    video.play();
                </script>
                <button onclick="play()">Play</button>
                <button onclick="pause()">Pause</button>
                <script type="text/javascript">
                    var timestamp = {timestamp}
                    var window_size = {window_size}

                    function play(){{
                        var video= document.getElementById("window");
                        if (video.currentTime < timestamp || video.currentTime >= timestamp + window_size){{
                            video.currentTime = timestamp;
                        }}
                        video.play()
                        setInterval(function(){{
                            if(video.currentTime >= timestamp + window_size){{
                                video.currentTime = timestamp;
                            }}
                        }},1);
                        <!--video.playbackRate = 2-->
                    }}
                
                    function pause(){{
                        var video= document.getElementById("window");
                        video.pause();
                    }}
                </script>
            </body>
        '''))

display_html('Timer2.mp4', 4, 2)